In [1]:
# Dependecies
import pandas as pd
from os import environ
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.types import Integer, Text, String, Float, DateTime

# Import CSVs as DFs
song_rank_df = pd.read_csv("./Resources/song_data.csv")
spotify_features_df = pd.read_csv('./Resources/spotify_features_df_new.csv')

song_rank_df

,Position,Track Name,Artist,Streams,Date,Region,new_URL
0,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493,2017-01-01,us,4Km5HrUvYTaSUfiSGPJeQR
1,2,Fake Love,Drake,1180074,2017-01-01,us,343YBumqHu19cGoGARUTsd
2,3,Starboy,The Weeknd,1064351,2017-01-01,us,5aAx2yezTd8zXrkmtKl66Z
3,4,Closer,The Chainsmokers,1010492,2017-01-01,us,7BKLCZ1jbUBVqRi2FVlTVw
4,5,Black Beatles,Rae Sremmurd,874289,2017-01-01,us,6fujklziTHa8uoM5OQSfIo
...,...,...,...,...,...,...,...
74195,196,Portland,Drake,184537,2018-01-09,us,2bjwRfXMk4uRgOD9IBYl9h
74196,197,All On Me,Devin Dawson,184519,2018-01-09,us,2mfahQ0EaaZWq4cFNg6A1o
74197,198,From The D To The A (feat. Lil Yachty),Tee Grizzley,184428,2018-01-09,us,2NyrXRn4tancYPW6JwtTl2
74198,199,Now Or Never,Halsey,184394,2018-01-09,us,7i2DJ88J7jQ8K7zqFX2fW8


In [2]:
spotify_features_df

,genre,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,new_artist_name
0,Movie,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.6110,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,Henri Salvador
1,Movie,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.2460,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816,Martin & les fées
2,Movie,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.9520,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368,Joseph Williams
3,Movie,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.7030,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,Henri Salvador
4,Movie,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.9500,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390,Fabien Nataf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176769,Soul,Quickly (feat. Brandy),1U0OMWvR89Cm20vCNar50f,39,0.2310,0.736,222667,0.701,0.000000,A#,0.2030,-4.345,Minor,0.1000,99.991,4/4,0.770,John Legend
176770,Soul,P.O.P.,2gGqKJWfWbToha2YmDxnnj,43,0.1040,0.802,201173,0.516,0.000485,D,0.1050,-9.014,Major,0.2130,175.666,4/4,0.482,Belly
176771,Soul,I'll Take Care Of You - Single Version,2iZf3EUedz9MPqbAvXdpdA,32,0.5660,0.423,144667,0.337,0.000000,A#,0.2760,-13.092,Minor,0.0436,80.023,4/4,0.497,"Bobby ""Blue"" Bland"
176772,Soul,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.0329,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969,Jr Thomas & The Volcanos


In [3]:
# Import URI connection
from config import database_uri

In [4]:
#db_uri = environ.get(database_uri)
engine = create_engine(database_uri)

In [5]:
# Create Table for spotify features

table1 = 'spotifyfeatures'

spotify_features_df.to_sql(
    table1,
    engine,
    if_exists='replace',
    index=False,
    dtype={
        "genre": String,
        "artist_name": String,
        "track_name": String,
        "track_id": String,
        "popularity": Integer,
        "acousticness": Float,
        "danceability": Float,
        "duration_ms": Integer,
        "energy": Float,
        "instrumentalness": Float,
        "key": String,
        "liveness": Float,
        "loudness": Float,
        "mode": String,
        "speechiness": Float,
        "tempo": Float,
        "time_signature": String,
        "valence": Float
    }
)

In [6]:
# Create song data table

table2= 'songdata'

song_rank_df.to_sql(
    table2,
    engine,
    if_exists='replace',
    index=False,
    dtype={
        "position": Integer,
        "track_name": String,
        "artist": String,
        "streams": Integer,
        "date": DateTime,
        "region": String,
        "new_URL": String
    }
)